In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# from keras import backend as K

import tensorflow as tf

In [ ]:
# including Root Mean Square Error function, required for report

# loss function from demo code
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

In [ ]:
# include weather data from South Alabama 
# Locate weather data path and create data set
WeatherDataPath = r"C:\Users\Peter Chapman\Personal_Research_Projects\weather-parameters-prediction-ML\LSTM Weather Code\Project 5 Mesonet Dataset.csv"
WeatherData = pd.read_csv(
    WeatherDataPath,
    parse_dates=['Hour'],
    index_col='Hour'
)

# second variable of interest: humidity
# relative humidity at 2 meters, which is typically used for weather reports
HumidityData = WeatherData[['RH_2m']]